# Contradiction

`Propósito`: uma linguagem construída da forma mais contra-intuitiva possível para qualquer programador experiente, afim de trazer a tona as suas habilidades de aprender uma lógica nova, sem ter o benefício de ter visto coisas similares.

## Gramática

Definição geral da limguagem.

```
<program> ::= <definitions> <commands

<definitions> ::= <definition> ||
                        <definition> <definitions>
                        
<definition> ::= int id ;  ||
                        : float id ;

<abstraction> ::= lambda id expression ;

<commands> ::= <command> ||
                    <command> <commands>
                    
<command> ::= id  = <expression> ;

<application> ::= id expression ;

<expression> ::= id ||
                    number ||
                    <expression> <expression> +
                    <expression> <expression> -
                    <expression> <expression> *
                    <expression> <expression> /
```


## Código teste

In [281]:
bCode = "float x? float y? x = 1? y = x + 2?"
# result:
# x = 1
# y = -1

In [282]:
mCode = "float x? float y? float z? return mais10(a) sizeof a + 10? x = 1? y = x + 2? z = mais10(y)?"
# result
# x = 1
# y = -1
# z = -11

In [283]:
hCode = "float x? float y? float z? return mul2(a) sizeof a*2? x = 4? if (x < 10 ) y = 1? else y = 2? z = mul2(y-x)?"

In [284]:
prg = "int x; int a = 10; def mais10(y) : y+10; x = mais10(a);"

### Analisador Léxico

In [285]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('MINUS', r'\+')
lg.add('PLUS', r'-')
lg.add('DIV', r'\*')
lg.add('MUL', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'float')
lg.add('FLOAT', r'int')

lg.add('IF', r'if')
lg.add('ELSE', r'else')
lg.add('WHILE', r'while')
lg.add('FOR', r'for')
lg.add('SEP', r'\?')
lg.add("INDENTATION", r'sizeof')
lg.add("DEF", r"return")

lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r'=')
lg.ignore('\s+')

lexer = lg.build()

In [286]:
tokens = lexer.lex(hCode)
print(list(tokens))

[Token('INT', 'float'), Token('ID', 'x'), Token('SEP', '?'), Token('INT', 'float'), Token('ID', 'y'), Token('SEP', '?'), Token('INT', 'float'), Token('ID', 'z'), Token('SEP', '?'), Token('DEF', 'return'), Token('ID', 'mul2'), Token('OPEN_PARENS', '('), Token('ID', 'a'), Token('CLOSE_PARENS', ')'), Token('INDENTATION', 'sizeof'), Token('ID', 'a'), Token('DIV', '*'), Token('NUMBER', '2'), Token('SEP', '?'), Token('ID', 'x'), Token('EQUALS', '='), Token('NUMBER', '4'), Token('SEP', '?'), Token('IF', 'if'), Token('OPEN_PARENS', '('), Token('ID', 'x'), Token('COMP', '<'), Token('NUMBER', '10'), Token('CLOSE_PARENS', ')'), Token('ID', 'y'), Token('EQUALS', '='), Token('NUMBER', '1'), Token('SEP', '?'), Token('ELSE', 'else'), Token('ID', 'y'), Token('EQUALS', '='), Token('NUMBER', '2'), Token('SEP', '?'), Token('ID', 'z'), Token('EQUALS', '='), Token('ID', 'mul2'), Token('OPEN_PARENS', '('), Token('ID', 'y'), Token('PLUS', '-'), Token('ID', 'x'), Token('CLOSE_PARENS', ')'), Token('SEP', '?')]

### Árvore sintática

In [287]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id, tp, expr=None):
        self.id = id
        self.tp = tp
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)

class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1

    def accept(self, visitor):
        visitor.visit_while(self)

class For(BaseBox):
    def __init__(self, idinic, exprinic, expr1, comp, expr2, idincr, exprincr, ie1):
        self.idinic=idinic
        self.exprinic=exprinic
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.idincr=idincr
        self.exprincr=exprincr
        self.ie1=ie1
    
    def accept(self, visitor):
        visitor.visit_for(self)

class DefFunc(BaseBox):
    def __init__(self, id, param, expr):
        self.id = id
        self.param = param
        self.expr = expr
    
    def accept(self, visitor):
        visitor.visit_deffunc(self)

class Func(BaseBox):
    def __init__(self, id, param):
        self.id = id
        self.param = param
    
    def accept(self, visitor):
        return visitor.visit_func(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        return visit(self)

class Func(Expr):
    def __init__(self, id, param):
        self.id = id
        self.param = param

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value

class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    pass


class Sub(BinaryOp):
    pass


class Mul(BinaryOp):
    pass


class Div(BinaryOp):
    pass


### Analisador Sintático

In [288]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    ['NUMBER',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT','FLOAT', 'ID','SEP',
     'EQUALS','COMP','IF','ELSE','WHILE','FOR', 'DEF', 'INDENTATION', 'OPEN_PARENS', 'CLOSE_PARENS'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : INT ID SEP')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int", None)

@pg.production('vardecl : FLOAT ID SEP')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "float", None)

@pg.production('vardecl : INT ID EQUALS expression SEP')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int", p[3])

@pg.production('vardecl : FLOAT ID EQUALS expression SEP')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "float", p[3])

@pg.production('vardecl : DEF ID OPEN_PARENS ID CLOSE_PARENS INDENTATION expression SEP')
def vardecl_function(p):
    return DefFunc(p[1].getstr(), f"__{p[3].getstr()}__", p[6])


##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEP')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('openstatement : FOR OPEN_PARENS ID EQUALS expression SEP expression COMP expression SEP ID EQUALS expression CLOSE_PARENS openstatement')
def statement_while(p):
    return For (p[2].getstr(),p[4],p[6],p[7],p[8],p[10].getstr(),p[12],p[14])


##################################################
# COMANDOS - CASO FECHADO
##################################################


@pg.production('closedstatement : ID EQUALS expression SEP')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('closedstatement : FOR OPEN_PARENS ID EQUALS expression SEP expression COMP expression SEP ID EQUALS expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return For (p[2].getstr(),p[4],p[6],p[7],p[8],p[10].getstr(),p[12],p[14])


##################################################
# Expression
##################################################

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : ID OPEN_PARENS expression CLOSE_PARENS')
def expression_func(p):
    return Func(p[0].getstr(), p[2])

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [289]:
arvore = parser.parse(lexer.lex(hCode))

In [290]:
class Visitor(object):
    pass

### Geração da tabela de simbolos

In [291]:
VARIABLES = {}
FUNCTIONS = {}
ST = {}

class SymbolTable(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)

    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)

    def visit_vardecl(self, d):
        ST[d.id] = d.tp
        VARIABLES[d.id] = 0

    def visit_deffunc(self, func):
        FUNCTIONS[func.id] = func
        VARIABLES[func.param] = 0
        ST[func.id] = "func"

In [292]:
arvore.accept(SymbolTable())
print(ST)
print(VARIABLES)
print(FUNCTIONS)

{'x': 'int', 'y': 'int', 'z': 'int', 'mul2': 'func'}
{'x': 0, 'y': 0, 'z': 0, '__a__': 0}
{'mul2': <__main__.DefFunc object at 0x00000289D60C8E50>}


### Decorator

In [293]:
class Decorator(Visitor):
    def visit_prog(self, p):
        p.decls.accept(self)
        p.stmts.accept(self)
    
    def visit_vardecls(self, d):
        d.decl.accept(self)
        if d.decls!=None:
          d.decls.accept(self)
    
    def visit_vardecl(self, d):
        if d.expr:
            d.decor_type=ST[d.id]
            d.expr.accept(self)
    
    def visit_deffunc(self, f):
        pass

    def visit_func(self, f):
        f.decor_type = "int"
        f.param.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
            d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)
    
    def visit_atrib(self, i):
        if i.id in ST:
            i.decor_type=ST[i.id]
        else:
            raise AssertionError(f'{i.id} not declared')
        i.expr.accept(self)
    
    def visit_ifelse(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
        if i.ie2!=None:
            i.ie2.accept(self)
    
    def visit_for(self, i):
        i.atrib1.accept(self)
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.atrib2.accept(self)
        i.ie1.accept(self)

    def visit_while(self, i):
        i.expr1.accept(self)
        i.expr2.accept(self)
        i.ie1.accept(self)
    
    def visit_id(self, i):
        if i.value in ST:
            i.decor_type=ST[i.value]
        else:
            raise AssertionError(f'id: {i.value} not declared')

    def visit_number(self, i):
        i.decor_type= "int" 
    
    def binaryOpVisit(self, a):
        a.left.accept(self)
        a.right.accept(self)
        a.decor_type = a.left.decor_type

    def visit_add(self, a):
        self.binaryOpVisit(a)
    
    def visit_sub(self, a):
        self.binaryOpVisit(a)

    def visit_mul(self, a):
        self.binaryOpVisit(a)

    def visit_div(self, a):
        self.binaryOpVisit(a)

In [294]:
arvore.accept(Decorator())

### Verificador de tipos

In [295]:
# VISITOR - TYPE VERIFIER

class TypeVerifier(Visitor):
    def visit_prog(self, i):
        i.stmts.accept(self)

    def visit_statements(self, d):
        d.stmt.accept(self)
        if d.stmts!=None:
          d.stmts.accept(self)

    def visit_statement(self, d):
        d.cmd.accept(self)

    def visit_atrib(self, i):
        if i.decor_type!=i.expr.decor_type:
          raise AssertionError('type error')
  
    def visit_ifelse(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_while(self, i):
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

    def visit_for(self, i):
        # verificação da inicialização do for
        if i.inicdecor_type!=i.exprinic.decor_type:
          raise AssertionError('type error')

        # verificação da comaração do for
        if i.expr1.decor_type!=i.expr2.decor_type:
          raise AssertionError('type error')

        # verificação da atualização do for
        if i.incrdecor_type!=i.exprincr.decor_type:
          raise AssertionError('type error')


In [296]:
arvore.accept(TypeVerifier())

In [297]:
class Eval(Visitor):
    def visit_prog(self, prog):
        prog.decls.accept(self)
        if prog.stmts:
            prog.stmts.accept(self)
    
    def visit_vardecls(self, v):
        v.decl.accept(self)
        if v.decls!=None:
            v.decls.accept(self)
    
    def visit_vardecl(self, v): 
        if v.expr:
            number = v.expr.accept(self)
            VARIABLES[v.id] = float(number) if ST[v.id] == "float" else int(number)

    def visit_deffunc(self, f):
        pass

    def visit_func(self, call):
        if call.id in FUNCTIONS:
            function = FUNCTIONS.get(call.id)
            VARIABLES[function.param] = call.param.accept(self)
            value = function.expr.accept(self)
            del VARIABLES[function.param]
            return value
        raise AssertionError(f'{call.id} not declared')

    def visit_statements(self, s):
        s.stmt.accept(self)
        if s.stmts!=None:
            s.stmts.accept(self)

    def visit_statement(self, s):
        s.cmd.accept(self)
    
    def visit_while(self, d):
        expr1 = d.expr1.accept(self)
        expr2 = d.expr2.accept(self)
        if(eval(f"{expr1} {d.comp} {expr2}")):
            d.ie1.accept(self)

    def visit_for(self, d):
        number = d.exprinic.accept(self)
        VARIABLES[d.idinic] = float(number) if ST[d.idinic] == "float" else int(number)

        expr1 = d.expr1.accept(self)
        expr2 = d.expr2.accept(self)
        while(eval(f"{expr1} {d.comp} {expr2}")):
            d.ie1.accept(self)
            number = d.exprincr.accept(self)
            VARIABLES[d.idincr] = float(number) if ST[d.idincr] == "float" else int(number)
    
    def visit_ifelse(self, d):
        expr1 = d.expr1.accept(self)
        expr2 = d.expr2.accept(self)
        if(eval(f"{expr1} {d.comp} {expr2}")):
            d.ie1.accept(self)
        elif d.ie2!=None:
            d.ie2.accept(self)
    
    def visit_atrib(self, d):
        number = d.expr.accept(self)
        VARIABLES[d.id] = float(number) if ST[d.id] == "float" else int(number)

    def visit_id(self, i):
        if f'__{i.value}__' in VARIABLES:
            return VARIABLES[f'__{i.value}__']
        elif i.value in VARIABLES:
            return VARIABLES[i.value]
        raise AssertionError(f'{i.value} not declared')

    def visit_add(self, o):
        return o.left.accept(self) + o.right.accept(self)
    
    def visit_sub(self, o):
        return o.left.accept(self) - o.right.accept(self)
    
    def visit_mul(self, o):
        return o.left.accept(self) * o.right.accept(self)

    def visit_div(self, o):
        return o.left.accept(self) / o.right.accept(self)
    
    def visit_number(self, n):
        return n.value

In [298]:
arvore.accept(Eval())

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
arvore=parser.parse(lexer.lex(hCode))
arvore.accept(SymbolTable())
arvore.accept(Decorator())
arvore.accept(TypeVerifier())
arvore.accept(Eval())

SyntaxError: invalid syntax (<string>, line 1)

In [299]:
print(hCode)

float x? float y? float z? return mul2(a) sizeof a*2? x = 4? if (x < 10 ) y = 1? else y = 2? z = mul2(y-x)?


In [ ]:
print(VARIABLES)

{'x': 1, 'y': -1, 'z': -11}
